# Fairness statistics analysis on Adult dataset

## Task 1

We have two populations Blue (privileged) and Red (unprivileged), with the Blue population being 9 times larger than the Red population.

Individuals from both populations are requesting to attend XAI training to improve competency in this important area. Number of places is limited. The administrators of the training have decided to give priority to enrolling individuals who may need this training in the future, although unfortunately it is difficult to predict who will benefit.

The decision rule adopted:
1. In the Red group, half of the people will find the skills useful in future and half will not. Administrators randomly allocate 50% of people to training.
2. in the Blue group, 80% of people will find the training useful in future and 20% will not, although of course it is not known who will find it useful. The administrators have built a predictive model based on user behaviour in predicting for whom it will be useful and whom will not. The model has the following performance:


| Blue                     	| Will use XAI 	| Will not use XAI 	| Total 	|
|--------------------------	|--------------	|------------------	|-------	|
| Enrolled in training     	| 60           	| 5               	| 65    	|
| not enrolled in training 	| 20            	| 15               	| 35    	|
| Total                    	| 80           	| 20               	| 100   	|


Task: Calculate the Demographic parity, equal opportunity and predictive rate parity coefficients for this decision rule.

Starred task: How can this decision rule be changed to improve its fairness?

### Solution
![a](tex/task1.png)

## Task 2

### 2. For the selected protected attribute (age, gender, race) calculate the following fairness coefficients: Statistical parity, Equal opportunity, Predictive parity.

![a](images/2_0.png)

Only Statistical parity of the 3 coefficients does not satisfy inequality: 80% < . < 125%. The Statistical parity shows that the model is indeed biased and predicts higher income for males.

### 3. Train another model (different hyperparameters, feature transformations etc.) and see how the coefficients Statistical parity, Equal opportunity, Predictive parity behave for it.

|  |  |
| --- | --- |
| ![a](images/3_0.png) | ![a](images/3_1.png) |

As we can see on the above data frames, the Statistical parity, Equal opportunity, Predictive parity behave similarly to the model in the 2 point (RandomForestClassifier).

### 4. Apply the selected bias mitigation technique on the first model. Check how Statistical parity, Equal opportunity, Predictive parity coefficients behave after this correction.

![a](images/4_0.png)

I removed gender related variables that contain information about the gender: gender, relationship. As we can see this did not help. The model receives similar statistics as before. This suggests that the information about gender may be highly correlated with much more variables and it may be hard to remove this information from the input data. To overcome this issue one may try another mitigation techniques.

### 5. Compare the quality (performance) of the three models with their fairness coefficients. Is there any correlation?

It is hard to see any clear correlation as all metrics (fairness coefficients and performance (auc)) are similar across different models. However, we can see that the GradientBoostingClassifier has significantly lower Statistical parity coefficient (0.26) than other (> 0.28) and significantly better auc (0.925) than others (< 0.911). This behavior is not suprising as the model that would better model the correlations (and therefore biases) in the dataset should perform better.

## Appendix

Source code

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import dalex as dx
import warnings
import plotly.io as pio

pio.renderers.default = "notebook"
warnings.simplefilter(action='ignore', category=FutureWarning)
np.random.seed(42)
OUTPUT_COLUMN = "income"

pd.read_csv("adult.csv")

/home/ppawlik-asus/eXplainableMachineLearning-2023/xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [2]:

scaler = StandardScaler()

def get_preprocessed_dataset():
    df = pd.read_csv("adult.csv")
    df = pd.get_dummies(df, columns=["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country"])
    df[OUTPUT_COLUMN] = (df[OUTPUT_COLUMN] == ">50K")

    X, y = df.drop(columns=[OUTPUT_COLUMN]), df[OUTPUT_COLUMN]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42
    )

    X_train[:] = scaler.fit_transform(X_train)
    X_test[:] = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_preprocessed_dataset()

def inverse_scaler(column_values, variable: str):
    variable_idx = X_train.columns.get_loc(variable)
    return column_values * scaler.scale_[variable_idx] + scaler.mean_[variable_idx]

### 1. Train a model for the selected dataset.


In [3]:
models = [RandomForestClassifier(), GradientBoostingClassifier(), LogisticRegression()]
y_hats = []

for m in models:
    m.fit(X_train, y_train)
    y_hats.append(m.predict_proba(X_test))

### 2. For the selected protected attribute (age, gender, race) calculate the following fairness coefficients: Statistical parity, Equal opportunity, Predictive parity.


In [4]:
def task2(model, X=X_test):
    explainer = dx.Explainer(model, X, y_test, verbose=False)
    fobject = explainer.model_fairness(
        protected=X_test["gender_Male"].apply(lambda x: "male" if x > 0 else "female"),
        privileged="male"
    )

    fobject.fairness_check(verbose=False)

    df = fobject.result[["STP", "TPR", "PPV"]].rename(columns={"STP": "Statistical parity", "TPR": "Equal opportunity", "PPV": "Predictive parity"})
    df["auc"] = explainer.model_performance().result["auc"].item()
    df.columns = pd.MultiIndex.from_product([[str(model)], df.columns])
    return df


task2(models[0])


/home/ppawlik-asus/eXplainableMachineLearning-2023/xai/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  0.816944  1.124848  0.975275  0.245283  0.297794


RandomForestClassifier()                                              
             Statistical parity Equal opportunity Predictive parity       auc
female                 0.297794          0.816944          0.975275  0.906748
male                   1.000000          1.000000          1.000000  0.906748

### 3. Train another model (different hyperparameters, feature transformations etc.) and see how the coefficients Statistical parity, Equal opportunity, Predictive parity behave for it.


In [5]:
task2(models[1])

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  0.800312  1.114014  1.068268  0.150685  0.263374


/home/ppawlik-asus/eXplainableMachineLearning-2023/xai/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names



GradientBoostingClassifier()                                      \
                 Statistical parity Equal opportunity Predictive parity   
female                     0.263374          0.800312          1.068268   
male                       1.000000          1.000000          1.000000   

                  
             auc  
female  0.925975  
male    0.925975

In [6]:
task2(models[2])

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC      PPV       FPR       STP
female  0.833871  1.136585  1.04749  0.189474  0.281746


/home/ppawlik-asus/eXplainableMachineLearning-2023/xai/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



LogisticRegression()                                              
         Statistical parity Equal opportunity Predictive parity       auc
female             0.281746          0.833871           1.04749  0.911023
male               1.000000          1.000000           1.00000  0.911023

### 4. Apply the selected bias mitigation technique on the first model. Check how Statistical parity, Equal opportunity, Predictive parity coefficients behave after this correction.


In [7]:
models.append(RandomForestClassifier())

columns_gender_related = ["gender_Male", "gender_Female", "relationship_Husband", "relationship_Wife"]

models[-1].fit(X_train.drop(columns=columns_gender_related), y_train)
y_hats.append(models[-1].predict_proba(X_test.drop(columns=columns_gender_related)))

df = task2(models[-1], X_test.drop(columns=columns_gender_related))
df = df["RandomForestClassifier()"]
df.columns = pd.MultiIndex.from_product([["RandomForestClassifier() - gender variable removed"], df.columns])
df

/home/ppawlik-asus/eXplainableMachineLearning-2023/xai/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  0.779789  1.122276  0.978112  0.230769  0.284133


RandomForestClassifier() - gender variable removed                    \
                                       Statistical parity Equal opportunity   
female                                           0.284133          0.779789   
male                                             1.000000          1.000000   

                                    
       Predictive parity       auc  
female          0.978112  0.906311  
male            1.000000  0.906311

### 5. Compare the quality (performance) of the three models with their fairness coefficients. Is there any correlation?


### 6. ! COMMENT on the results obtained in (2)-(5)